# Hyper Parameter Tuning
- hyper parameter: 모델 설정과 관련해 직접 지정할 수 있는 매개변수
- model parameter: 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수

최적의 파라미터를 찾는거기 때문에 cross_validation과 다르게 train_test_split을 하지 않고 모델을 수행함

### GridSearchCV

In [1]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 로드
iris_input, iris_target = load_iris(return_X_y=True)

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 값
params = {
    'n_neighbors': range(1,13,2)
}

# 첫 번째 인자: 모델
# 두 번째 인자: 테스트 할 파라미터 (딕셔너리)
# scoring: 평가 지표 (accuracy, precision, recall, f1)
# cv: 반복 횟수
grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

print('최적의 파라미터: ', grid.best_params_)
print('최적의 모델 객체: ', grid.best_estimator_)
print('최적화된 점수: ', grid.best_score_)

최적의 파라미터:  {'n_neighbors': 7}
최적의 모델 객체:  KNeighborsClassifier(n_neighbors=7)
최적화된 점수:  0.9800000000000001


In [2]:
best_knn = grid.best_estimator_
best_knn.score(iris_input, iris_target)

0.9733333333333334

### RandomSearchCV
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다.
    - 탐색범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다.
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다.

In [3]:
from sklearn.model_selection import RandomizedSearchCV

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 생성
params = {
    'n_neighbors': range(1,100,2) # RandomSearchCV는 랜덤하게 뽑아서 범위를 넓게 줘도 괜찮음 / but, GridSearchCV는 아님
}

# n_iter: 탐색할 최적의 하이퍼 파라미터 조합 수 (기본 값: 10)
#         값이 크면 시간이 오래 걸림 / 값이 작으면 좋은 조합을 찾을 가능성 저하
rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0) 
rd_search.fit(iris_input, iris_target)

print('최적의 파라미터: ', rd_search.best_params_) # gridsearchcv에서는 최적의 파라미터로 7을 반환했지만 randomsearchcv에서는 5를 최적으로 뽑아냄
print('최적의 모델 객체: ', rd_search.best_estimator_)
print('최적화된 점수: ', rd_search.best_score_)
rd_search.cv_results_ # 탐색한 모든 조합에 대한 정보를 담고 있음

최적의 파라미터:  {'n_neighbors': 5}
최적의 모델 객체:  KNeighborsClassifier()
최적화된 점수:  0.9733333333333334


{'mean_fit_time': array([0.00119815, 0.00140171, 0.00059896, 0.00019784, 0.00040321,
        0.00019851, 0.0004003 , 0.00021815, 0.00019951, 0.00060072]),
 'std_fit_time': array([0.00039913, 0.00049355, 0.00048905, 0.00039568, 0.00049386,
        0.00039701, 0.00049027, 0.00026718, 0.00039902, 0.00049049]),
 'mean_score_time': array([0.00400081, 0.00439744, 0.00360212, 0.02951035, 0.00212598,
        0.00179935, 0.00939856, 0.00616865, 0.00180335, 0.00179958]),
 'std_score_time': array([0.00167372, 0.0010249 , 0.00101777, 0.04602013, 0.00113664,
        0.00040042, 0.00049303, 0.00029389, 0.00040211, 0.00040097]),
 'param_n_neighbors': masked_array(data=[57, 23, 21, 83, 5, 55, 77, 63, 45, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'n_neighbors': 57},
  {'n_neighbors': 23},
  {'n_neighbors': 21},
  {'n_neighbors': 83},
  {'n_neighbors': 5},
  {'n_neighbors': 55},
  {'n_neighb

---

### HyperOpt

- 하이퍼파라미터 최적화를 위한 라이브러리
- 탐색 공간이 클수록 효율적이며, GridSearchCV는 탐색 공간이 작을수록 효율적
- hyperopt는 베이지안 최적화를 사용해 더 적은 횟수로 최적의 값을 찾을 수 있음

**hyper.hp클래스**
<table border="1">
  <thead>
    <tr>
      <th>함수명</th>
      <th>설명</th>
      <th>사용 방법</th>
      <th>예시 코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>hp.uniform</td>
      <td>연속적인 실수 값 샘플링</td>
      <td>hp.uniform(label, low, high)</td>
      <td><code>hp.uniform('learning_rate', 0.01, 0.1)</code></td>
    </tr>
    <tr>
      <td>hp.quniform</td>
      <td>연속적이지만 일정 간격(q)을 갖는 값 샘플링</td>
      <td>hp.quniform(label, low, high, q)</td>
      <td><code>hp.quniform('num_layers', 1, 5, 1)</code></td>
    </tr>
    <tr>
      <td>hp.loguniform</td>
      <td>로그 스케일로 분포된 실수 값 샘플링</td>
      <td>hp.loguniform(label, low, high)</td>
      <td><code>hp.loguniform('reg_param', -3, 0)</code></td>
    </tr>
    <tr>
      <td>hp.randint</td>
      <td>정수 값 샘플링</td>
      <td>hp.randint(label, upper)</td>
      <td><code>hp.randint('num_trees', 1, 100)</code></td>
    </tr>
    <tr>
      <td>hp.choice</td>
      <td>주어진 리스트 중 임의의 값 샘플링</td>
      <td>hp.choice(label, options)</td>
      <td><code>hp.choice('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
    </tr>
    <tr>
      <td>hp.normal</td>
      <td>정규분포에서 값 샘플링</td>
      <td>hp.normal(label, mean, std)</td>
      <td><code>hp.normal('dropout_rate', 0.3, 0.05)</code></td>
    </tr>
    <tr>
      <td>hp.lognormal</td>
      <td>로그 정규분포에서 값 샘플링</td>
      <td>hp.lognormal(label, mean, std)</td>
      <td><code>hp.lognormal('scale', 0, 1)</code></td>
    </tr>
  </tbody>
</table>

In [4]:
# !pip install hyperopt

In [5]:
from hyperopt import hp

# 검색할 공간 만들기
search_space = {
    'x': hp.quniform('x', -10, 10, 1),      # -10에서 10까지 그 간격이 1인 정수 집합
    'y': hp.quniform('y', -15, 15, 1)       # -15에서 15까지 그 간격이 1인 정수 집합
}

In [6]:
import hyperopt

# 목적 함수
def objective(search_space):
    x = search_space['x']
    y = search_space['y']

    return {
        'loss': x**2 + 20 * y,
        'status': hyperopt.STATUS_OK
    }

In [7]:
from hyperopt import fmin, tpe, Trials

# 탐색 과정을 저장하는 객체
trials = Trials()

# fmin(): 목적 함수의 최소값을 찾는 함수
best_val = fmin(
    fn=objective,           # 목적함수
    space=search_space,     # 검색공간: 최소값에 해당하는 것을 찾아줌
    algo=tpe.suggest,      # 베이지안 최적화 적용
    max_evals=500,           # 반복 횟수
    trials=trials           # 탐색과정 저장
)

best_val

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 500/500 [00:05<00:00, 84.07trial/s, best loss: -300.0] 


{'x': np.float64(-0.0), 'y': np.float64(-15.0)}

In [8]:
# 탐색 과정 -> 목적 함수의 반환값 (loss와 실행 상태) 저장
trials.results
trials.vals

{'x': [np.float64(-5.0),
  np.float64(-4.0),
  np.float64(3.0),
  np.float64(8.0),
  np.float64(3.0),
  np.float64(-2.0),
  np.float64(-0.0),
  np.float64(7.0),
  np.float64(-2.0),
  np.float64(5.0),
  np.float64(9.0),
  np.float64(-5.0),
  np.float64(-4.0),
  np.float64(1.0),
  np.float64(-7.0),
  np.float64(3.0),
  np.float64(-1.0),
  np.float64(8.0),
  np.float64(10.0),
  np.float64(-2.0),
  np.float64(-9.0),
  np.float64(1.0),
  np.float64(2.0),
  np.float64(6.0),
  np.float64(-8.0),
  np.float64(-2.0),
  np.float64(5.0),
  np.float64(0.0),
  np.float64(-6.0),
  np.float64(-4.0),
  np.float64(-3.0),
  np.float64(1.0),
  np.float64(2.0),
  np.float64(4.0),
  np.float64(-10.0),
  np.float64(-1.0),
  np.float64(-6.0),
  np.float64(-3.0),
  np.float64(4.0),
  np.float64(1.0),
  np.float64(1.0),
  np.float64(7.0),
  np.float64(2.0),
  np.float64(-1.0),
  np.float64(4.0),
  np.float64(5.0),
  np.float64(3.0),
  np.float64(9.0),
  np.float64(6.0),
  np.float64(-3.0),
  np.float64(-0.0),
 

- hyperopt를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [9]:
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=42)

# 1. 검색 공간 : 최적의 하이퍼파라미터를 구하는 것이니 하이퍼파라미터를 검색 공간에 작성해야 함
search_space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100),     # quniform: 정수 값
    'max_depth': hp.quniform('max_depth', 3, 10, 1),              
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),        # uniform: 실수 값
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

# 2. 목적 함수
def xgb_objective(ss):

    xgb_clf = XGBClassifier(
        n_estimators=int(ss['n_estimators']),     # 정수형
        max_depth=int(ss['max_depth']),           # 정수형
        learning_rate=ss['learning_rate'],
        colsample_bytree=ss['colsample_bytree']
    )

    mean_acc = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()  # 
    return {
        'loss': -1 *  mean_acc,     # fmin 최적화알고리즘은 최소값을 찾아주는 거니까 -1을 곱해서 성능을 확인할 수 있음?
        'status': hyperopt.STATUS_OK
    }

# 3. Trials() + fmin()
trials = Trials()
best = fmin(
    fn=xgb_objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)
best

100%|██████████| 50/50 [00:16<00:00,  3.01trial/s, best loss: -0.971830985915493] 


{'colsample_bytree': np.float64(0.5490817956639423),
 'learning_rate': np.float64(0.18474750398584688),
 'max_depth': np.float64(7.0),
 'n_estimators': np.float64(500.0)}

---

### Optuna

- 자동화된 최적화(Automated Optimization)를 위한 최신 라이브러리

<table border="1">
    <thead>
        <tr>
            <th>함수명</th>
            <th>설명</th>
            <th>사용 방법</th>
            <th>예시 코드</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>suggest_uniform</td>
            <td>연속적인 실수 값 샘플링</td>
            <td>trial.suggest_uniform(name, low, high)</td>
            <td><code>trial.suggest_uniform('learning_rate', 0.01, 0.1)</code></td>
        </tr>
        <tr>
            <td>suggest_discrete_uniform</td>
            <td>연속적이지만 일정 간격(step)을 갖는 값 샘플링</td>
            <td>trial.suggest_discrete_uniform(name, low, high, step)</td>
            <td><code>trial.suggest_discrete_uniform('num_layers', 1, 5, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_loguniform</td>
            <td>로그 스케일로 분포된 실수 값 샘플링</td>
            <td>trial.suggest_loguniform(name, low, high)</td>
            <td><code>trial.suggest_loguniform('reg_param', 1e-3, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_int</td>
            <td>정수 값 샘플링</td>
            <td>trial.suggest_int(name, low, high, step)</td>
            <td><code>trial.suggest_int('num_trees', 1, 100)</code></td>
        </tr>
        <tr>
            <td>suggest_categorical</td>
            <td>주어진 리스트 중 임의의 값 샘플링</td>
            <td>trial.suggest_categorical(name, choices)</td>
            <td><code>trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
        </tr>
        <tr>
            <td>suggest_float</td>
            <td>연속적인 실수 값 샘플링 (<code>step</code> 사용 가능)</td>
            <td>trial.suggest_float(name, low, high, step=None, log=False)</td>
            <td><code>trial.suggest_float('alpha', 0.1, 1.0, step=0.1)</code></td>
        </tr>
    </tbody>
</table>

In [10]:
# !pip install optuna

In [11]:
import optuna

# 목적 함수
def objective(trial):
    x = trial.suggest_uniform('x', -10,10)
    y = trial.suggest_uniform('y', -15,15)
    return (x-3)**2 + (y+5)**2

# 스터디 생성
study = optuna.create_study(direction='minimize')

# 최적화 실행
study.optimize(objective, n_trials=500)

# 결과 확인
print(study.best_value)
print(study.best_params)

[I 2025-03-28 14:08:46,540] A new study created in memory with name: no-name-ef1ffb8f-1002-456c-9b51-5d4d7ef57fd4
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6600\1247780634.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10,10)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6600\1247780634.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  y = trial.suggest_uniform('y', -15,15)
[I 2025-03-28 14:08:46,542] Trial 0 finished with value: 407.00118793024603 and parameters: {'x': -2.6162154272483047, 'y': 14.376772491955006}. Best is trial 0 with value: 407.00118793024603.
[I 2025-03-28 14:08:46,543] Trial 1 finished with value: 314.67342070232274 and parameters: {'x': 2

0.0010646921511730802
{'x': 3.031599770179085, 'y': -5.0081330606663235}


In [12]:
import optuna.visualization as vis

vis.plot_param_importances(study).show()        # plotly: 동적 그래프를 만들어줌

In [13]:
vis.plot_optimization_history(study)

- optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [14]:
# 1. 목적함수
def xgb_optuna_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    xgb_clf = XGBClassifier(**params)   # params 전달
    return cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()

# 2. study 객체 -> 최적화
study = optuna.create_study(direction='maximize')
study.optimize(xgb_optuna_objective, n_trials=50)

# 3. 결과 출력
print(study.best_params)
print(study.best_value)

[I 2025-03-28 14:22:10,568] A new study created in memory with name: no-name-a51d493c-4692-46dc-9e47-bfaae7ae883c
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6600\1067317825.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-03-28 14:22:10,829] Trial 0 finished with value: 0.9577464788732394 and parameters: {'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.11896101664783323, 'colsample_bytree': 0.7791066389110717}. Best is trial 0 with value: 0.9577464788732394.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6600\1067317825.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-03-28 14:22:11,188] Trial 1 finished with value: 0.960093896713615 and parameters: {'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.12797340001044416, 'colsample_bytree': 0.6320706396693168}. Best is trial 1 with value: 0.960

{'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.11469957404482106, 'colsample_bytree': 0.529936357874034}
0.971830985915493


In [15]:
from sklearn.metrics import accuracy_score

xgb_hpopt = XGBClassifier(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.18474750398584688,
    colsample_bytree=0.5490817956639423
)

xgb_optuna = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.11469957404482106,
    colsample_bytree=0.529936357874034
)

xgb_hpopt.fit(X_train, y_train)
xgb_optuna.fit(X_train, y_train)

hpopt_pred = xgb_hpopt.predict(X_test)
optuna_pred = xgb_optuna.predict(X_test)

print(f'HyperOpt 최적 파라미터 적용: {accuracy_score(y_test, hpopt_pred)}')
print(f'Optuna 최적 파라미터 적용: {accuracy_score(y_test, optuna_pred)}')

HyperOpt 최적 파라미터 적용: 0.965034965034965
Optuna 최적 파라미터 적용: 0.958041958041958
